# English - Alice in wonderland

## Data preparation

In [1]:
from importlib import reload

import preprocessor
preprocessor = reload(preprocessor)


pp = preprocessor.Preprocessor("data/alice_in_wonderland.txt", "eng")
cleaned_corpus = pp.process()
pp.split(cleaned_corpus)

## Subword segmentation

In [2]:
import sentpiece
sentpiece = reload(sentpiece)


NUM_CHARS = 44
TRAIN_DATA_PATH = "data/eng_train.txt"
MODEL_NAME = "eng_model_char"
SEG_DATA_PATH = "en_s1.txt"


sentpiece.train_model(TRAIN_DATA_PATH, MODEL_NAME, NUM_CHARS)

In [3]:
!mv $MODEL_NAME* models/

In [4]:
sentpiece.segmentation(TRAIN_DATA_PATH, MODEL_NAME, SEG_DATA_PATH)

## Train LM

In [8]:
TRAIN_SEG_DATA_PATH = "segmented_corpora/en_s1.txt"

!bash train_script.sh TRAIN_SEG_DATA_PATH "data/eng_test.txt" 40 3

rm: cannot remove 'rnnlm/models/model': No such file or directory
rm: cannot remove 'rnnlm/models/model.output.txt': No such file or directory
debug mode: 2
train file: data/eng_train.txt
valid file: data/eng_test.txt
class size: 9999
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: model
Starting training using file data/eng_train.txt
Vocab size: 4119
Words in train file: 24231
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 9.3558    Words/sec: 540.6   VALID entropy: 8.1441
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 8.8022    Words/sec: 524.0   VALID entropy: 7.7922
Iter:   2	Alpha: 0.100000	   TRAIN entropy: 8.4398    Words/sec: 530.1   VALID entropy: 7.6852
Iter:   3	Alpha: 0.100000	   TRAIN entropy: 8.1320    Words/sec: 515.8   VALID entropy: 7.4599
Iter:   4	Alpha: 0.100000	   TRAIN entropy: 7.8664    Words/sec: 617.0   VALID entropy: 7.3073
Iter:   5	Alpha: 0.100000	   TRAIN entropy: 7.6337    Words/sec: 733.8   VALID entropy: 7.2368
Iter:   6	Alpha: 0.100000	   TRAIN entropy

In [9]:
!mv model model.output.txt rnnlm/models/

## Data generation

In [ ]:
GEN_DATA_PATH = "gen_data/eng_model_chr/"

!bash gen_script.sh GEN_DATA_PATH

# Bengali

# Analysis